In [1]:
import time

def znajdz_najdluzsze_slowo(lista_slow):
  naj_dl_slowo = None
  naj_dl_slowo_len = 0
  for slowo in lista_slow:
    if len(slowo) > naj_dl_slowo_len:
      naj_dl_slowo = slowo
      naj_dl_slowo_len = len(slowo)
  return naj_dl_slowo


lista_slow = ['hadoop', 'yarn', 'mapreduce']
start = time.process_time()
print(znajdz_najdluzsze_slowo(lista_slow))
print(time.process_time() - start)

mapreduce
0.00023437000000003927


In [2]:
lista_slow_dluga = lista_slow*10
start = time.process_time()
print(znajdz_najdluzsze_slowo(lista_slow_dluga))
print(time.process_time() - start)

mapreduce
0.000809567000000122


In [3]:
import multiprocessing
multiprocessing.cpu_count()

8

In [12]:
dane = ['hadoop', 'yarn', 'mapreduce', 'hadoop2', 'yarn2', 'mapreduce2']
# map zwraca iterator, który możemy przerobić na listę, aby podejrzeć
list(map(len, dane))

[6, 4, 9, 7, 5, 10]

In [11]:
from functools import reduce
from multiprocessing import Pool

# reducer dostaje dwie krotki, z których pod indeksem 1 mamy długość słowa
# zwraca krotkę, która zawiera większą długość 
def reducer(krotka_a, krotka_b):
  if krotka_a[1] > krotka_b[1]:
    return krotka_a
  return krotka_b

# chcemy podzielić nasze dane na mniejsze porcje równej wielkości
def chunkify(dane, liczba_chunks):
  # slicing naszych danych - od indeksu i do końca, ale co 2 słowo (liczba_chunks to nasz step value dla slicingu)
  return(dane[i::liczba_chunks] for i in range(liczba_chunks))

# mapujemy nasze dane w 2 krokach
def chunks_mapper(chunk):
  # funckja map bierze jako argument nazwę funkcji oraz obiekt, po którym możemy iterować (np. lista, krotka, etc.)
  # i zwraca iterator, gdybyśmy wydrukowali ten iterator (najpierw trzeba go zamienić na listę list(mapped_chunk))
  # to zwróci nam listę długości słów w naszej porcji danych
  mapped_chunk = map(len, chunk)
  # print(list(mapped_chunk))
  # print(list(mapped_chunk))
  # drugi krok łączy w krotki odpowiednio słowa z porcji danych i ich długość funkcją zip
  mapped_chunk = zip(chunk, mapped_chunk)
  # print(list(mapped_chunk))
  # na koniec zwracamy zredukowaną porcję danych do jednej krotki z najdłuższym wyrazem i jego długością
  return reduce(reducer, mapped_chunk)



start = time.process_time()
# data_chunks = chunkify(lista_slow_dluga, 2)
data_chunks = chunkify(open('./pd/opowiesc.txt').read().split(), 2)
pool = Pool(2)

# krok 1
# w tym kroku dostaniemy iterator, który po zamianie na listę zwróci listę krotek z najdłuższymi słowami i idpowiadającymi im długościami
mapped = pool.map(chunks_mapper, data_chunks)
# krok 2
# w tym kroku redukujemy nasze krotki -> zwracamy tylko jedną, która ma najdłuższy wyraz
reduced = reduce(reducer, mapped)

print(reduced)
print(time.process_time() - start)

('round-the-world,', 16)
0.020023502999999998


In [6]:
data_chunks = chunkify(lista_slow_dluga, 2)
list(map(chunks_mapper, data_chunks))

[('mapreduce', 9), ('mapreduce', 9)]